## Merge Fundamental + Momentum + Tecnico


In [1]:
import nasdaqdatalink as ndl
import pandas as pd
from ta import add_all_ta_features
import pandas as pd
import zipfile
import json

In [81]:
#configurations
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


## Importamos datasets a mergear

In [2]:
rebalance_returns = pd.read_csv('../data/rebalance_returns.csv', parse_dates=['rebalance_date'])
tecnico = pd.read_csv('../data/tecnico.csv',          parse_dates=['rebalance_date'])
fundamental = pd.read_csv("../data/fundamental.csv",       parse_dates=["date"])
sp500 = pd.read_csv("../data/sp500.csv", parse_dates=["date"])

In [3]:


# 1. Crecimientos YoY para variables fundamentales
for col in ['revenue', 'ebitda','ebit', 'eps', 'equity', 'capex', 'pe', 'pb', 'fcfps', 'ncfo']:
    fundamental[f'{col}_yoy'] = fundamental.groupby('ticker')[col].pct_change()

C:\Users\Ignacio\AppData\Local\Temp\ipykernel_13508\1496921535.py:3: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  fundamental[f'{col}_yoy'] = fundamental.groupby('ticker')[col].pct_change()
C:\Users\Ignacio\AppData\Local\Temp\ipykernel_13508\1496921535.py:3: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  fundamental[f'{col}_yoy'] = fundamental.groupby('ticker')[col].pct_change()
C:\Users\Ignacio\AppData\Local\Temp\ipykernel_13508\1496921535.py:3: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Eit

In [4]:
snapshot_0331 = (
    sp500
      .sort_values(['ticker','date'])
      .reset_index(drop=True)
)


In [5]:
# extrae lista de (ticker, rebalance_date) únicos
tecnico = (
    tecnico
      .drop_duplicates()
      .sort_values(['ticker','rebalance_date'])
      .reset_index(drop=True)
)

snapshot_0331['date'] = pd.to_datetime(snapshot_0331['date'])
tecnico['rebalance_date'] = pd.to_datetime(tecnico['rebalance_date'])

# Ordena ambos DataFrames por 'ticker' y la columna de fecha
snapshot_0331 = snapshot_0331.sort_values(['date']).reset_index(drop=True)
tecnico = tecnico.sort_values(['rebalance_date']).reset_index(drop=True)

# para cada snapshot_0331.calendar_date toma el rebalance_date ≤ calendar_date
constituents_and_tecnico= pd.merge_asof(
    snapshot_0331,
    tecnico,
    left_on   = 'date',
    right_on  = 'rebalance_date',
    by        = 'ticker',
    direction = 'backward'
).rename(columns={'rebalance_date':'trade_date'})


In [6]:
fundamental = (
    fundamental
      .rename(columns={'date':'financial_year_end_date','datekey':'report_date'})
      .sort_values(['ticker','report_date'])
      .reset_index(drop=True)
)
fundamental['report_date'] = pd.to_datetime(fundamental['report_date'])
#fundamental['financial_year_end_date'] = pd.to_datetime(fundamental['financial_year_end_date'])


In [7]:
constituents_tecnico_fundamental = pd.merge_asof(
    constituents_and_tecnico.sort_values(['date']),
    fundamental.sort_values(['report_date']),
    left_on   = 'date',
    right_on  = 'report_date',
    by        = 'ticker',
    direction = 'backward'
)


In [8]:
merged = pd.merge(
    constituents_tecnico_fundamental.sort_values(['ticker', 'date']),
    rebalance_returns.sort_values(['ticker', 'rebalance_date']),
    left_on   = ['ticker', 'date'],
    right_on  = ['ticker', 'rebalance_date'],
    how='left'  # o 'inner'
)


### Lehmnan Brothers quiebra en 2008 y sale del indice

In [9]:
merged[merged['ticker'] == 'LEHMQ']

,date,ticker,year_x,trade_date,close_reb_adj,vol_1y,max_1y,min_1y,sma_1y,sma_2y,...,equity_yoy,capex_yoy,pe_yoy,pb_yoy,fcfps_yoy,ncfo_yoy,rebalance_date,closeadj,ret_6m,ret_12m
7515,1998-03-31,LEHMQ,1998.0,1998-03-31,17.443,1.884412e+08,17.443,11.532,14.137410,14.137410,...,NaN,NaN,NaN,NaN,NaN,NaN,1998-03-31,17.443,NaN,NaN
7516,1999-03-31,LEHMQ,1999.0,1999-03-31,13.920,1.263827e+09,19.802,5.766,13.279135,13.446403,...,-0.138469,0.630137,0.234587,-0.103528,-2.648109,-2.611147,1999-03-31,13.920,1.105900,-0.201972
7517,2000-03-31,LEHMQ,2000.0,2000-03-31,22.598,9.325708e+08,24.563,11.241,15.366945,14.318505,...,-0.192520,-0.745645,-0.227784,-0.409153,-1.425416,-1.423810,2000-03-31,22.598,0.661740,0.623420
7518,2001-03-31,LEHMQ,2001.0,2001-03-30,29.214,1.437350e+09,39.940,16.890,28.075321,21.691683,...,-0.080151,-0.785981,-0.113144,0.394188,-3.665601,-3.205869,2001-03-31,29.214,-0.151274,0.292769
7519,2002-03-31,LEHMQ,2002.0,2002-03-28,30.277,1.211465e+09,38.323,21.731,31.123540,29.563686,...,-0.054015,1.660714,-0.221832,0.053928,-0.771969,-0.726931,2002-03-31,30.277,0.143046,0.036387
7520,2003-03-31,LEHMQ,2003.0,2003-03-31,27.416,1.329330e+09,31.158,20.071,26.857822,28.967948,...,-0.321239,0.117517,0.019255,-0.174725,12.065608,8.603062,2003-03-31,27.416,0.186018,-0.094494
7521,2004-03-31,LEHMQ,2004.0,2004-03-31,39.823,1.317918e+09,43.009,28.461,34.583921,30.710149,...,-0.117024,0.124688,0.288499,0.047181,-1.184606,-1.233456,2004-03-31,39.823,0.205844,0.452546
7522,2005-03-31,LEHMQ,2005.0,2005-03-31,45.649,1.134938e+09,46.638,32.577,39.058124,36.816575,...,-0.111587,-0.019560,-0.033567,-0.216764,0.402274,0.456998,2005-03-31,45.649,0.186490,0.146297
7523,2006-03-31,LEHMQ,2006.0,2006-03-31,70.602,1.171674e+09,72.531,42.444,56.286870,47.706681,...,-0.124902,-0.302048,0.056164,-0.022897,-0.792166,-0.794150,2006-03-31,70.602,0.245163,0.546628
7524,2007-03-31,LEHMQ,2007.0,2007-03-30,68.991,1.107034e+09,84.308,58.160,71.002135,63.565796,...,-0.146687,-0.393375,0.374736,0.539661,-0.199376,-0.202193,2007-03-31,68.991,-0.047493,-0.022818


#### Seguimos teniendo 500 empresas aprox para cada ventana... excelente!

In [15]:
merged[merged['date'] == '2004-03-31']['ticker'].nunique()

500

In [16]:
merged.to_csv('../data/merged.csv', index=False)